In [1]:
%cd /userspace/pva/nsu-ai-taiga_version
%pwd

/userspace/pva/nsu-ai-taiga_version


/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/userspace/pva/nsu-ai-taiga_version'

In [43]:
import os
os.environ['ONLYFANS_CFG'] = '/userspace/pva/configs/lava_full.json'

In [70]:
from team_code.generate import setup_model_and_tokenizer

In [71]:
model, processor = setup_model_and_tokenizer()

2024-07-22 10:42:34 | INFO | team_code.generate | Current working directory: /userspace/pva/nsu-ai-taiga_version
2024-07-22 10:42:34 | INFO | team_code.generate | New working directory: /userspace/pva/nsu-ai-taiga_version/team_code/ONE-PEACE
2024-07-22 10:42:37 | INFO | team_code.ONE_PEACE.one_peace.tasks.base_task | dictionary: 50264 types
2024-07-22 10:43:31 | INFO | team_code.ONE_PEACE.one_peace.models.one_peace.one_peace_retrieval | logit_scale not exists, re-initialized
2024-07-22 10:43:34 | INFO | team_code.generate | ONE-PEACE model is loaded from the "/userspace/dra/nsu-ai/team_code/models/one-peace.pt".
2024-07-22 10:43:34 | INFO | team_code.generate | Restored working directory: /userspace/pva/nsu-ai-taiga_version
2024-07-22 10:43:34 | INFO | team_code.generate | The ONE-PEACE model is loaded.
/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when us

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-07-22 10:45:08 | INFO | team_code.generate | The large language model is loaded.
2024-07-22 10:45:28 | INFO | team_code.generate | The Ocr model is loaded.
2024-07-22 10:45:28 | INFO | team_code.generate | The YOLOv8 model is loaded.
/userspace/pva/miniconda3/envs/py9min/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2024-07-22 10:45:30 | INFO | team_code.generate | The Translation models are loaded.


In [72]:
from team_code.generate import load_images, MultimodalModel
import json
from peft import LoraConfig, get_peft_model
from transformers import LlavaNextProcessor
from torch.utils.data import Dataset
import torch
from transformers import Trainer, TrainingArguments
import sys
import importlib

In [111]:
class Llava_finetuning_Dataset(Dataset):
    def __init__(self, json_file: str, processor: LlavaNextProcessor):
        with open(json_file, 'r') as f:
            self.data = json.load(f)
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        item = self.data[idx]
        result = {'text': item['conversations'][0]['value'], 'image': item['image'], 'answer': item['conversations'][1]['value']}
        print(result)

        return result

In [112]:

class Collator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, samples):
        print(samples)
        images = [item['image'] for item in samples]
        human_texts = [item['text'] for item in samples]
        gpt_texts = [item['answer'] for item in samples]

        loaded_images= load_images(images)
        
        p_inputs = []
        for prompt, image in zip(human_texts, loaded_images):
            p_inputs.append(self.processor(text=prompt, images=image, return_tensors="pt"))

        return {'input': p_inputs, 'label': gpt_texts}

In [113]:
def train(model: MultimodalModel, processor: LlavaNextProcessor, batch_size: int, epochs: int, dir_train_dataset='train_dataset.json', dir_eval_dataset='eval_dataset.json'):
    

    # modules = find_all_linear_names(model.llm) 
    train_dataset = Llava_finetuning_Dataset(json_file=dir_train_dataset, processor=processor)

    eval_dataset = Llava_finetuning_Dataset(json_file=dir_eval_dataset, processor=processor)
    collator = Collator(processor)

    # Configuration
    
    # lora_config = LoraConfig.from_peft_type(**model.llm.peft_config)
    lora_config = model.llm.peft_config

    # llm_model = get_peft_model(model.llm, lora_config)

    training_args = TrainingArguments(
        output_dir="/userspace/pva/train/results",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="/userspace/pva/train/logs",
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model.llm,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=collator,
    )
    # Train the model
    trainer.train()

In [115]:
train_json_path, eval_dataset = "/userspace/pva/train/train.json", "/userspace/pva/train/validation.json"
train(model, processor, 1, 10, train_json_path, eval_dataset)

{'default': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/userspace/pva/weights/llava_next', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=8, target_modules={'base_layer', 'default'}, lora_alpha=32, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)}
{'text': '<s>[INST] You are a useful and friendly assistant with great erudition and developed intelligence. You can keep up a conversation on various topics and even know how to play complex intellectual games. I have just looked at an <image> that probably corresponds to the following text description. a black and white photo of a living room with a couch, chair, and table. One of Pahlmann’s most outstanding model rooms was 

KeyError: 'image'